In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import json
import random 
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Activation ,Dropout
from keras.optimizers import gradient_descent_v2

lemmatizer=WordNetLemmatizer()
intents=json.loads(open('inputs.json').read())
words=[]
classes =[]
documents =[]
ignore = ['?',',','.','!',':'] 
#Data pre processing
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore]
words=sorted(set(words))
classes=sorted(set(classes))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))
training = []
output_empty = [0]*len(classes)
for document in documents:
    bag=[]
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row=list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag,output_row])
random.shuffle(training)
training = np.array(training)
train_data, test_data = train_test_split(training, test_size=0.2, random_state=42)

# Convert the training and test sets back to lists
train_x = list(train_data[:, 0])
train_y = list(train_data[:, 1])
test_x = list(test_data[:, 0])
test_y = list(test_data[:, 1])
#Neural Network model
model= Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))
sgd = gradient_descent_v2.SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
hist=model.fit(np.array(train_x),np.array(train_y),epochs=250,batch_size=5,verbose=1)
model.save('chatbot.model')
model.save('chatbot.h5',hist)

Epoch 1/250


C:\Users\95814\AppData\Local\Temp\ipykernel_16092\987877566.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
d:\python_package\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


10/10 [==============================] - 1s 2ms/step - loss: 2.8411 - accuracy: 0.0000e+00
Epoch 2/250
10/10 [==============================] - 0s 2ms/step - loss: 2.7153 - accuracy: 0.1489
Epoch 3/250
10/10 [==============================] - 0s 2ms/step - loss: 2.6441 - accuracy: 0.1489
Epoch 4/250
10/10 [==============================] - 0s 2ms/step - loss: 2.6095 - accuracy: 0.1064
Epoch 5/250
10/10 [==============================] - 0s 3ms/step - loss: 2.5326 - accuracy: 0.2553
Epoch 6/250
10/10 [==============================] - 0s 2ms/step - loss: 2.5581 - accuracy: 0.1915
Epoch 7/250
10/10 [==============================] - 0s 2ms/step - loss: 2.5354 - accuracy: 0.1915
Epoch 8/250
10/10 [==============================] - 0s 2ms/step - loss: 2.4086 - accuracy: 0.3191
Epoch 9/250
10/10 [==============================] - 0s 2ms/step - loss: 2.3367 - accuracy: 0.2553
Epoch 10/250
10/10 [==============================] - 0s 2ms/step - loss: 2.3129 - accuracy: 0.3617
Epoch 11/250
10/1

In [2]:
#Finding accuracy
train_loss, train_accuracy = model.evaluate(np.array(train_x), np.array(train_y), verbose=1)
print(f'Training loss: {train_loss:.4f}')
print(f'Training accuracy: {train_accuracy:.4f}')


test_loss, test_accuracy = model.evaluate(np.array(test_x), np.array(test_y), verbose=1)
print(f'Test loss: {test_loss:.4f}')
print(f'Test accuracy: {test_accuracy:.4f}')

2/2 [==============================] - 0s 8ms/step - loss: 0.1166 - accuracy: 0.9362
Training loss: 0.1166
Training accuracy: 0.9362
1/1 [==============================] - 0s 24ms/step - loss: 1.8142 - accuracy: 0.5000
Test loss: 1.8142
Test accuracy: 0.5000


In [3]:
import numpy as np
import json
import random 
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow import keras
from keras.models import load_model
lemmatizer=WordNetLemmatizer()
intents=json.loads(open('inputs.json').read())
words=pickle.load(open('words.pkl','rb'))
classes=pickle.load(open('classes.pkl','rb'))
model=load_model('chatbot.model')
#creating functions for the model to search for patterns and tags in inputs to create relevant responses
def clean_up(line):
    line_words = nltk.word_tokenize(line)
    line_words = [lemmatizer.lemmatize(word) for word in line_words]
    return line_words
def bag_words(line):
    line_words=clean_up(line)
    bag=[0]*len(words)
    for i in line_words:
        for j,word in enumerate(words):
            if word==i:
                bag[j]=1
    return np.array(bag)
def predict_class(line):
    bag=bag_words(line)
    res=model.predict(np.array([bag]))[0]
    errort=0.25
    results =[[i,r] for i,r in enumerate(res) if r>errort]
    results.sort(key=lambda x:x[1],reverse = True)
    return_list=[]
    for r in results:
        return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    return return_list
def reponses(intents_list,intents_json):
    tag=intents_list[0]['intent']
    listintents=intents_json['intents']
    for i in listintents:
        if i['tag']==tag:
            result =random.choice(i['responses'])
            break
    return result



    


In [4]:
import os
import playsound
from googletrans import Translator, constants
from datetime import datetime
from gtts import gTTS
#Taking user inputs and then generating output for user.
print("---------------------Ruby is now Online ------------------------")
Name=input("May I know your name?")
print("Hey {0} this is Ruby from VIT-Bhopal".format(Name))
InputLanguageChoice = input("Which language do you want to continue?\n To continue in English enter en \n To continue in Telugu enter te \n To continue in Hindi hi")
    


while True:
    
    ques = input("Query:")
    translator = Translator()
    Translation = translator.translate(ques,dest="en",src=InputLanguageChoice)
    query=Translation.text
    #query = input("{name} :".format(name=Name))
    if ques == 'Bye' or ques=="bye" or ques =='ok thanks':
      
      print("'Ruby:' Bye")
      
      
      print("For more information please visit: https://admission.vitbhopal.ac.in/")
      break
    else:  
      ints=predict_class(query)
      res=reponses(ints,intents)
      translator =Translator()
      translation = translator.translate(res, dest=InputLanguageChoice)
      print(f"'Ruby:'{translation.text}")
      tts=gTTS(text=translation.text,lang =InputLanguageChoice)
      date_string = datetime.now().strftime("%d%m%Y%H%M%S")
      file = "testel"+date_string+".mp3"
      tts.save(file)
      playsound.playsound(file)
      os.remove(file)
    
    





    


---------------------Ruby is now Online ------------------------
Hey User this is Ruby from VIT-Bhopal
'Ruby:'Yes! my name is Ruby-The virtual assistant of VIT Bhopal Admissions department.
'Ruby:'Have a nice day
'Ruby:'Hi nice to see you again
'Ruby:'We appreciate your interest kindly attempt of VITEEE examination or your JEE score can also be used.
'Ruby:' Bye
For more information please visit: https://admission.vitbhopal.ac.in/
